In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from joblib import Parallel, delayed
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [13]:
df = pd.read_csv(r"Train_data.csv")
df.head()
#, encoding='ISO-8859-1'

duration protocol_type   service flag  src_bytes  dst_bytes  land  \
0         0           tcp  ftp_data   SF        491          0     0   
1         0           udp     other   SF        146          0     0   
2         0           tcp   private   S0          0          0     0   
3         0           tcp      http   SF        232       8153     0   
4         0           tcp      http   SF        199        420     0   

   wrong_fragment  urgent  hot  ...  dst_host_same_srv_rate  \
0               0       0    0  ...                    0.17   
1               0       0    0  ...                    0.00   
2               0       0    0  ...                    0.10   
3               0       0    0  ...                    1.00   
4               0       0    0  ...                    1.00   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                    0.03                         0.17   
1                    0.60                         0.88   
2                    0.05                         0.00   
3                    0.00                         0.03   
4                    0.00                         0.00   

   dst_host_srv_diff_host_rate  dst_host_serror_rate  \
0                         0.00                  0.00   
1                         0.00                  0.00   
2                         0.00                  1.00   
3                         0.04                  0.03   
4                         0.00                  0.00   

   dst_host_srv_serror_rate  dst_host_rerror_rate  dst_host_srv_rerror_rate  \
0                      0.00                  0.05                      0.00   
1                      0.00                  0.00                      0.00   
2                      1.00                  0.00                      0.00   
3                      0.01                  0.00                      0.01   
4                      0.00                  0.00                      0.00   

    attack  Last_flag  
0   normal         20  
1   normal         15  
2  neptune         19  
3   normal         21  
4   normal         21  

[5 rows x 43 columns]

In [16]:
print("Attack types:") 
print(df["attack"].unique())

print("protocol types:") 
print(df["protocol_type"].unique())

print("service types:") 
print(df["service"].unique())

print("flag types:") 
print(df["flag"].unique())

Attack types:
['normal' 'neptune' 'warezclient' 'ipsweep' 'portsweep' 'teardrop' 'nmap'
 'satan' 'smurf' 'pod' 'back' 'guess_passwd' 'ftp_write' 'multihop'
 'rootkit' 'buffer_overflow' 'imap' 'warezmaster' 'phf' 'land'
 'loadmodule' 'spy' 'perl']
protocol types:
['tcp' 'udp' 'icmp']
service types:
['ftp_data' 'other' 'private' 'http' 'remote_job' 'name' 'netbios_ns'
 'eco_i' 'mtp' 'telnet' 'finger' 'domain_u' 'supdup' 'uucp_path' 'Z39_50'
 'smtp' 'csnet_ns' 'uucp' 'netbios_dgm' 'urp_i' 'auth' 'domain' 'ftp'
 'bgp' 'ldap' 'ecr_i' 'gopher' 'vmnet' 'systat' 'http_443' 'efs' 'whois'
 'imap4' 'iso_tsap' 'echo' 'klogin' 'link' 'sunrpc' 'login' 'kshell'
 'sql_net' 'time' 'hostnames' 'exec' 'ntp_u' 'discard' 'nntp' 'courier'
 'ctf' 'ssh' 'daytime' 'shell' 'netstat' 'pop_3' 'nnsp' 'IRC' 'pop_2'
 'printer' 'tim_i' 'pm_dump' 'red_i' 'netbios_ssn' 'rje' 'X11' 'urh_i'
 'http_8001' 'aol' 'http_2784' 'tftp_u' 'harvest']
flag types:
['SF' 'S0' 'REJ' 'RSTR' 'SH' 'RSTO' 'S1' 'RSTOS0' 'S3' 'S2' 'OTH']


In [17]:
print(df.dtypes)

duration                         int64
protocol_type                   object
service                         object
flag                            object
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
num_compromised                  int64
root_shell                       int64
su_attempted                     int64
num_root                         int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
num_outbound_cmds                int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate                    float64
 srv_serror_rate         

In [18]:
print(df["protocol_type"].unique())

['tcp' 'udp' 'icmp']


In [19]:
protocol_mapping = {'tcp': 1, 'udp': 2, 'icmp': 3}

# Replace the values using the mapping
df['protocol_type'] = df['protocol_type'].replace(protocol_mapping)

In [20]:
print(df["protocol_type"].unique())

[1 2 3]


In [21]:
print(df["service"].unique())

['ftp_data' 'other' 'private' 'http' 'remote_job' 'name' 'netbios_ns'
 'eco_i' 'mtp' 'telnet' 'finger' 'domain_u' 'supdup' 'uucp_path' 'Z39_50'
 'smtp' 'csnet_ns' 'uucp' 'netbios_dgm' 'urp_i' 'auth' 'domain' 'ftp'
 'bgp' 'ldap' 'ecr_i' 'gopher' 'vmnet' 'systat' 'http_443' 'efs' 'whois'
 'imap4' 'iso_tsap' 'echo' 'klogin' 'link' 'sunrpc' 'login' 'kshell'
 'sql_net' 'time' 'hostnames' 'exec' 'ntp_u' 'discard' 'nntp' 'courier'
 'ctf' 'ssh' 'daytime' 'shell' 'netstat' 'pop_3' 'nnsp' 'IRC' 'pop_2'
 'printer' 'tim_i' 'pm_dump' 'red_i' 'netbios_ssn' 'rje' 'X11' 'urh_i'
 'http_8001' 'aol' 'http_2784' 'tftp_u' 'harvest']


In [22]:
service_mapping = {
    'ftp_data': 1, 'other': 2, 'private': 3, 'http': 4, 'remote_job': 5, 'name': 6,
    'netbios_ns': 7, 'eco_i': 8, 'mtp': 9, 'telnet': 10, 'finger': 11, 'domain_u': 12,
    'supdup': 13, 'uucp_path': 14, 'Z39_50': 15, 'smtp': 16, 'csnet_ns': 17, 'uucp': 18,
    'netbios_dgm': 19, 'urp_i': 20, 'auth': 21, 'domain': 22, 'ftp': 23, 'bgp': 24, 'ldap': 25,
    'ecr_i': 26, 'gopher': 27, 'vmnet': 28, 'systat': 29, 'http_443': 30, 'efs': 31, 'whois': 32,
    'imap4': 33, 'iso_tsap': 34, 'echo': 35, 'klogin': 36, 'link': 37, 'sunrpc': 38, 'login': 39,
    'kshell': 40, 'sql_net': 41, 'time': 42, 'hostnames': 43, 'exec': 44, 'ntp_u': 45,
    'discard': 46, 'nntp': 47, 'courier': 48, 'ctf': 49, 'ssh': 50, 'daytime': 51, 'shell': 52,
    'netstat': 53, 'pop_3': 54, 'nnsp': 55, 'IRC': 56, 'pop_2': 57, 'printer': 58, 'tim_i': 59,
    'pm_dump': 60, 'red_i': 61, 'netbios_ssn': 62, 'rje': 63, 'X11': 64, 'urh_i': 65,
    'http_8001': 66, 'aol': 67, 'http_2784': 68, 'tftp_u': 69, 'harvest': 70
}

# Replace the values using the mapping
df['service'] = df['service'].replace(service_mapping)


In [23]:
print(df["service"].unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70]


In [24]:
df["flag"].unique()

array(['SF', 'S0', 'REJ', 'RSTR', 'SH', 'RSTO', 'S1', 'RSTOS0', 'S3',
       'S2', 'OTH'], dtype=object)

In [25]:
flag_mapping = {'SF': 1, 'S0': 2, 'REJ': 3, 'RSTR': 4, 'SH': 5, 'RSTO': 6,
                'S1': 7, 'RSTOS0': 8, 'S3': 9, 'S2': 10, 'OTH': 11}

# Replace the values using the mapping
df['flag'] = df['flag'].replace(flag_mapping)

In [26]:
print(df["flag"].unique())

[ 1  2  3  4  5  6  7  8  9 10 11]


In [27]:
df["attack"].unique()

array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl'], dtype=object)

In [28]:
attack_mapping = {'normal': 1, 'neptune': 2, 'warezclient': 3, 'ipsweep': 4, 'portsweep': 5, 'teardrop': 6,
                  'nmap': 7,'satan': 8, 'smurf': 9, 'pod': 10, 'back': 11, 'guess_passwd': 12, 'ftp_write': 13,
                  'multihop': 14,'rootkit': 15, 'buffer_overflow': 16, 'imap': 17, 'warezmaster': 18, 'phf': 19, 
                  'land': 20, 'loadmodule': 21, 'spy': 22, 'perl': 23}

# Replace the values using the mapping
df['attack'] = df['attack'].replace(attack_mapping)

In [29]:
df["attack"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23], dtype=int64)

In [30]:
df.dtypes

duration                         int64
protocol_type                    int64
service                          int64
flag                             int64
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
num_compromised                  int64
root_shell                       int64
su_attempted                     int64
num_root                         int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
num_outbound_cmds                int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate                    float64
 srv_serror_rate         

In [31]:
df.to_csv("Preprocessed_data.csv",index=False)